# Natural Language Processing

![NLP](https://s3.amazonaws.com/codecademy-content/courses/NLP/Natural_Language_Processing_Overview.gif "NLP")

Resources: 

https://www.codecademy.com/learn/natural-language-processing/modules/getting-started-with-nlp-module/cheatsheet

### 1 - Text Preprocessing

Common tasks include:

Noise removal — stripping text of formatting (e.g., HTML tags).

Tokenization — breaking text into individual words.

Normalization — cleaning text data in any other way:

Stemming is a blunt axe to chop off word prefixes and suffixes. “booing” and “booed” become “boo”, but “sing” may become “s” and “sung” would remain “sung.”

Lemmatization is a scalpel to bring words down to their root forms. For example, NLTK’s savvy lemmatizer knows “am” and “are” are related to “be.”

Other common tasks include lowercasing, stopwords removal, spelling correction, etc.

In [9]:
# 1 - Text Processing

# regex for removing punctuation!
import re
# nltk preprocessing magic
import nltk
from nltk.corpus import wordnet
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Create Part of Speech Function

def get_part_of_speech(word):
      probable_part_of_speech = wordnet.synsets(word)
      pos_counts = Counter()
      pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
      pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
      pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
      pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )

      most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
      return most_likely_part_of_speech

text = "So many squids are jumping out of suitcases these days that you can barely go anywhere without seeing one burst forth from a tightly packed valise. I went to the dentist the other day, and sure enough I saw an angry one jump out of my dentist's bag within minutes of arriving. She hardly even noticed."

cleaned = re.sub('\W+', ' ', text)
tokenized = word_tokenize(cleaned)

stemmer = PorterStemmer()
stemmed = [stemmer.stem(token) for token in tokenized]

## -- CHANGE these -- ##
lemmatizer = WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]

print("Stemmed text:")
print(stemmed)
print("\nLemmatized text:")
print(lemmatized)

Stemmed text:
['So', 'mani', 'squid', 'are', 'jump', 'out', 'of', 'suitcas', 'these', 'day', 'that', 'you', 'can', 'bare', 'go', 'anywher', 'without', 'see', 'one', 'burst', 'forth', 'from', 'a', 'tightli', 'pack', 'valis', 'I', 'went', 'to', 'the', 'dentist', 'the', 'other', 'day', 'and', 'sure', 'enough', 'I', 'saw', 'an', 'angri', 'one', 'jump', 'out', 'of', 'my', 'dentist', 's', 'bag', 'within', 'minut', 'of', 'arriv', 'she', 'hardli', 'even', 'notic']

Lemmatized text:
['So', 'many', 'squid', 'be', 'jump', 'out', 'of', 'suitcase', 'these', 'day', 'that', 'you', 'can', 'barely', 'go', 'anywhere', 'without', 'see', 'one', 'burst', 'forth', 'from', 'a', 'tightly', 'pack', 'valise', 'I', 'go', 'to', 'the', 'dentist', 'the', 'other', 'day', 'and', 'sure', 'enough', 'I', 'saw', 'an', 'angry', 'one', 'jump', 'out', 'of', 'my', 'dentist', 's', 'bag', 'within', 'minute', 'of', 'arrive', 'She', 'hardly', 'even', 'notice']


### 2 - Parsing Text

After Preprocessing, It may be helpful to know how the words relate to each other and the underlying syntax (grammar). Parsing is a stage of NLP concerned with segmenting text based on syntax.

You probably do not want to be doing any parsing by hand and NLTK has a few tricks up its sleeve to help you out:

Part-of-speech tagging (POS tagging) identifies parts of speech (verbs, nouns, adjectives, etc.). NLTK can do it faster (and maybe more accurately) than your grammar teacher.

Named entity recognition (NER) helps identify the proper nouns (e.g., “Natalia” or “Berlin”) in a text. This can be a clue as to the topic of the text and NLTK captures many for you.

Dependency grammar trees help you understand the relationship between the words in a sentence. It can be a tedious task for a human, so the Python library spaCy is at your service, even if it isn’t always perfect.

Regex parsing, using Python’s re library, allows for a bit more nuance. When coupled with POS tagging, you can identify specific phrase chunks. On its own, it can find you addresses, emails, and many other common patterns within large chunks of text.

In English we leave a lot of ambiguity, so syntax can be tough, even for a computer program.

In [14]:
# 2 - Parsing Text: words relate to each other and the underlying syntax (grammar)

# Parsing is Breaking up of text based on Grammar

squids_text = """So many squids are jumping out of suitcases these days. You can barely go anywhere without seeing one. 
I went to the dentist the other day. Sure enough, I saw an angry one jump out of my dentist's bag. 
She hardly even noticed."""

import spacy
from nltk import Tree

dependency_parser = spacy.load('en_core_web_sm')

parsed_squids = dependency_parser(squids_text)

# Assign my_sentence a new value:
my_sentence = "This is My New Sentence!"
my_parsed_sentence = dependency_parser(my_sentence)

def to_nltk_tree(node):
      if node.n_lefts + node.n_rights > 0:
        parsed_child_nodes = [to_nltk_tree(child) for child in node.children]
        return Tree(node.orth_, parsed_child_nodes)
      else:
        return node.orth_

for sent in parsed_squids.sents:
    to_nltk_tree(sent.root).pretty_print()
    
for sent in my_parsed_sentence.sents:
    to_nltk_tree(sent.root).pretty_print()

        jumping                
  _________|________________    
 |   |   squids    out      |  
 |   |     |        |       |   
 |   |    many      of     days
 |   |     |        |       |   
are  .     So   suitcases these

          go                       
  ________|______________________   
 |   |    |       |     without  | 
 |   |    |       |        |     |  
 |   |    |       |      seeing  . 
 |   |    |       |        |     |  
You can barely anywhere   one      
                                   

          went               
  _________|_________         
 |   |     to        |       
 |   |     |         |        
 |   |  dentist     day      
 |   |     |      ___|____    
 I   .    the   the     other

                    saw                          
  ___________________|_________________________   
 |   |    |              jump                  | 
 |   |    |      _________|__________          |  
 |   |    |     |    |    |         out        | 
 |   |    |   

### 3 - Language Models - Bag-of-Words Approach

How can we help a machine make sense of a bunch of word tokens? We can help computers make predictions about language by training a language model on a corpus (a bunch of example text).

Language models are probabilistic computer models of language. We build and use these models to figure out the likelihood that a given sound, letter, word, or phrase will be used. Once a model has been trained, it can be tested out on new texts.

One of the most common language models is the unigram model, a statistical language model commonly known as bag-of-words. As its name suggests, bag-of-words does not have much order to its chaos! What it does have is a tally count of each instance for each word.

Provided some initial preprocessing, bag-of-words would result in a mapping like:

{"the": 2, "squid": 1, "jump": 1, "out": 1, "of": 1, "suitcase": 1}

Now look at this sentence and mapping: “Why are your suitcases full of jumping squids?”

{"why": 1, "be": 1, "your": 1, "suitcase": 1, "full": 1, "of": 1, "jump": 1, "squid": 1}

You can see how even with different word order and sentence structures, “jump,” “squid,” and “suitcase” are shared topics between the two examples. Bag-of-words can be an excellent way of looking at language when you want to make predictions concerning topic or sentiment of a text. When grammar and word order are irrelevant, this is probably a good model to use.

![BOW](https://s3.amazonaws.com/codecademy-content/courses/NLP/bag-of-words.gif "BOW")

In [28]:
# 3 - Language Models

looking_glass_text = """
However, the egg only got larger and larger, and more and more human: when she had come within a few yards of it, she saw that it had eyes and a nose and mouth; and when she had come close to it, she saw clearly that it was HUMPTY DUMPTY himself. It cant be anybody else! she said to herself. Im as certain of it, as if his name were written all over his face.

It might have been written a hundred times, easily, on that enormous face. Humpty Dumpty was sitting with his legs crossed, like a Turk, on the top of a high wallsuch a narrow one that Alice quite wondered how he could keep his balanceand, as his eyes were steadily fixed in the opposite direction, and he didnt take the least notice of her, she thought he must be a stuffed figure after all.

And how exactly like an egg he is! she said aloud, standing with her hands ready to catch him, for she was every moment expecting him to fall.

Its very provoking, Humpty Dumpty said after a long silence, looking away from Alice as he spoke, to be called an eggVery!

I said you looked like an egg, Sir, Alice gently explained. And some eggs are very pretty, you know she added, hoping to turn her remark into a sort of a compliment.

Some people, said Humpty Dumpty, looking away from her as usual, have no more sense than a baby!

Alice didnt know what to say to this: it wasnt at all like conversation, she thought, as he never said anything to her; in fact, his last remark was evidently addressed to a treeso she stood and softly repeated to herself:

     Humpty Dumpty sat on a wall:
     Humpty Dumpty had a great fall.
     All the Kings horses and all the Kings men
     Couldnt put Humpty Dumpty in his place again.

That last line is much too long for the poetry, she added, almost out loud, forgetting that Humpty Dumpty would hear her.

Dont stand there chattering to yourself like that, Humpty Dumpty said, looking at her for the first time, but tell me your name and your business.

My name is Alice, but

Its a stupid enough name! Humpty Dumpty interrupted impatiently. What does it mean?

Must a name mean something? Alice asked doubtfully.

Of course it must, Humpty Dumpty said with a short laugh: my name means the shape I amand a good handsome shape it is, too. With a name like yours, you might be any shape, almost.

Why do you sit out here all alone? said Alice, not wishing to begin an argument.

Why, because theres nobody with me! cried Humpty Dumpty. Did you think I didnt know the answer to that? Ask another.

Dont you think youd be safer down on the ground? Alice went on, not with any idea of making another riddle, but simply in her good-natured anxiety for the queer creature. That wall is so very narrow!

What tremendously easy riddles you ask! Humpty Dumpty growled out. Of course I dont think so! Why, if ever I did fall offwhich theres no chance ofbut if I did Here he pursed his lips and looked so solemn and grand that Alice could hardly help laughing. If I did fall, he went on, The King has promised mewith his very own mouthtoto

To send all his horses and all his men, Alice interrupted, rather unwisely.

Now I declare thats too bad! Humpty Dumpty cried, breaking into a sudden passion. Youve been listening at doorsand behind treesand down chimneysor you couldnt have known it!

I havent, indeed! Alice said very gently. Its in a book.

Ah, well! They may write such things in a book, Humpty Dumpty said in a calmer tone. Thats what you call a History of England, that is. Now, take a good look at me! Im one that has spoken to a King, I am: mayhap youll never see such another: and to show you Im not proud, you may shake hands with me! And he grinned almost from ear to ear, as he leant forwards (and as nearly as possible fell off the wall in doing so) and offered Alice his hand. She watched him a little anxiously as she took it. If he smiled much more, the ends of his mouth might meet behind, she thought: and then I dont know what would happen to his head! Im afraid it would come off!

Yes, all his horses and all his men, Humpty Dumpty went on. Theyd pick me up again in a minute, they would! However, this conversation is going on a little too fast: lets go back to the last remark but one.

Im afraid I cant quite remember it, Alice said very politely.

In that case we start fresh, said Humpty Dumpty, and its my turn to choose a subject (He talks about it just as if it was a game! thought Alice.) So heres a question for you. How old did you say you were?

Alice made a short calculation, and said Seven years and six months.

Wrong! Humpty Dumpty exclaimed triumphantly. You never said a word like it!

I though you meant How old are you? Alice explained.

If Id meant that, Id have said it, said Humpty Dumpty. 
"""

In [29]:
# Create Part of Speech Function

#The part-of-speech function is used for lemmatization

from nltk.corpus import wordnet
from collections import Counter
def get_part_of_speech(word):
    probable_part_of_speech = wordnet.synsets(word)
    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )

    most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
    return most_likely_part_of_speech

In [30]:
# Bag-of-words can be an excellent way of looking at language when you want to make predictions concerning topic or sentiment 
# of a text. When grammar and word order are irrelevant, this is probably a good model to use.

# importing regex and nltk
import re, nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# importing Counter to get word counts for bag of words
from collections import Counter

# Change text to another string:
text = "This is a New Text file"

cleaned = re.sub('\W+', ' ', text).lower()
tokenized = word_tokenize(cleaned)

stop_words = stopwords.words('english')
filtered = [word for word in tokenized if word not in stop_words]

normalizer = WordNetLemmatizer()
normalized = [normalizer.lemmatize(token, get_part_of_speech(token)) for token in filtered]
# Comment out the print statement below
print(normalized)

# Define bag_of_looking_glass_words & print:

bag_of_looking_glass_words = Counter(normalized)
print(bag_of_looking_glass_words)

['new', 'text', 'file']
Counter({'new': 1, 'text': 1, 'file': 1})


### 4 - Language Models - N-Grams and NLM

For parsing entire phrases or conducting language prediction, you will want to use a model that pays attention to each word’s neighbors. Unlike bag-of-words, the n-gram model considers a sequence of some number (n) units and calculates the probability of each unit in a body of language given the preceding sequence of length n. Because of this, n-gram probabilities with larger n values can be impressive at language prediction.

Take a look at our revised squid example: “The squids jumped out of the suitcases. The squids were furious.”

A bigram model (where n is 2) might give us the following count frequencies:

{('', 'the'): 2, ('the', 'squids'): 2, ('squids', 'jumped'): 1, ('jumped', 'out'): 1, ('out', 'of'): 1, ('of', 'the'): 1, ('the', 'suitcases'): 1, ('suitcases', ''): 1, ('squids', 'were'): 1, ('were', 'furious'): 1, ('furious', ''): 1}

There are a couple problems with the n gram model:

How can your language model make sense of the sentence “The cat fell asleep in the mailbox” if it’s never seen the word “mailbox” before? During training, your model will probably come across test words that it has never encountered before (this issue also pertains to bag of words). A tactic known as language smoothing can help adjust probabilities for unknown words, but it isn’t always ideal.

For a model that more accurately predicts human language patterns, you want n (your sequence length) to be as large as possible. That way, you will have more natural sounding language, right? Well, as the sequence length grows, the number of examples of each sequence within your training corpus shrinks. With too few examples, you won’t have enough data to make many predictions.

Enter neural language models (NLM)! Much recent work within NLP has involved developing and training neural networks to approximate the approach our human brains take towards language. This deep learning approach allows computers a much more adaptive tack to processing human language.

In [32]:
# 4 - Language Models

# We Use the Above Looking Glass text for our Processing

import nltk, re
from nltk.tokenize import word_tokenize
# importing ngrams module from nltk
from nltk.util import ngrams
from collections import Counter

# It would be More Effective if Used for Very Large Texts

cleaned = re.sub('\W+', ' ', looking_glass_text).lower()
tokenized = word_tokenize(cleaned)

# Change the n value to 2:
looking_glass_bigrams = ngrams(tokenized, n=2)
looking_glass_bigrams_frequency = Counter(looking_glass_bigrams)

# Change the n value to 3:
looking_glass_trigrams = ngrams(tokenized, n=3)
looking_glass_trigrams_frequency = Counter(looking_glass_trigrams)

# Change the n value to a number greater than 3:
looking_glass_ngrams = ngrams(tokenized, n=5)
looking_glass_ngrams_frequency = Counter(looking_glass_ngrams)

print("Looking Glass Bigrams:")
print(looking_glass_bigrams_frequency.most_common(10))

print("\nLooking Glass Trigrams:")
print(looking_glass_trigrams_frequency.most_common(10))

print("\nLooking Glass n-grams:")
print(looking_glass_ngrams_frequency.most_common(10))

Looking Glass Bigrams:
[(('humpty', 'dumpty'), 19), (('dumpty', 'said'), 4), (('all', 'his'), 4), (('in', 'a'), 4), (('on', 'the'), 3), (('she', 'thought'), 3), (('as', 'he'), 3), (('said', 'humpty'), 3), (('horses', 'and'), 3), (('and', 'all'), 3)]

Looking Glass Trigrams:
[(('humpty', 'dumpty', 'said'), 4), (('said', 'humpty', 'dumpty'), 3), (('horses', 'and', 'all'), 3), (('when', 'she', 'had'), 2), (('she', 'had', 'come'), 2), (('it', 'she', 'saw'), 2), (('like', 'an', 'egg'), 2), (('looking', 'away', 'from'), 2), (('all', 'the', 'kings'), 2), (('that', 'humpty', 'dumpty'), 2)]

Looking Glass n-grams:
[(('all', 'his', 'horses', 'and', 'all'), 2), (('his', 'horses', 'and', 'all', 'his'), 2), (('horses', 'and', 'all', 'his', 'men'), 2), (('however', 'the', 'egg', 'only', 'got'), 1), (('the', 'egg', 'only', 'got', 'larger'), 1), (('egg', 'only', 'got', 'larger', 'and'), 1), (('only', 'got', 'larger', 'and', 'larger'), 1), (('got', 'larger', 'and', 'larger', 'and'), 1), (('larger', 'an

### 5 - Topic Models

We’ve touched on the idea of finding topics within a body of language. But what if the text is long and the topics aren’t obvious?

Topic modeling is an area of NLP dedicated to uncovering latent, or hidden, topics within a body of language. For example, one Codecademy curriculum developer used topic modeling to discover patterns within Taylor Swift songs related to love and heartbreak over time.

A common technique is to deprioritize the most common words and prioritize less frequently used terms as topics in a process known as term frequency-inverse document frequency (tf-idf). Say what?! This may sound counter-intuitive at first. Why would you want to give more priority to less-used words? Well, when you’re working with a lot of text, it makes a bit of sense if you don’t want your topics filled with words like “the” and “is.” The Python libraries gensim and sklearn have modules to handle tf-idf.

Whether you use your plain bag of words (which will give you term frequency) or run it through tf-idf, the next step in your topic modeling journey is often latent Dirichlet allocation (LDA). LDA is a statistical model that takes your documents and determines which words keep popping up together in the same contexts (i.e., documents). We’ll use sklearn to tackle this for us.

If you have any interest in visualizing your newly minted topics, word2vec is a great technique to have up your sleeve. word2vec can map out your topic model results spatially as vectors so that similarly used words are closer together. In the case of a language sample consisting of “The squids jumped out of the suitcases. The squids were furious. Why are your suitcases full of jumping squids?”, we might see that “suitcase”, “jump”, and “squid” were words used within similar contexts. This word-to-vector mapping is known as a word embedding.

In [36]:
# Enter the Texts for Topic Models

# Get Sherlock Homes texts in Python file in the Directory, it contains large Sherlock Homes Paragraphs
import os
print(os.listdir())

# Create Preprocessing Functions

import nltk, re
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

stop_words = stopwords.words('english')
normalizer = WordNetLemmatizer()

def get_part_of_speech(word):
    probable_part_of_speech = wordnet.synsets(word)
    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
    most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
    return most_likely_part_of_speech

# Used tin the Next Cell
def preprocess_text(text):
    cleaned = re.sub(r'\W+', ' ', text).lower()
    tokenized = word_tokenize(cleaned)
    normalized = [normalizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]
    filtered = [word for word in normalized if word not in stop_words]
    return " ".join(filtered)

['.ipynb_checkpoints', '1 - GETTING STARTED WITH NATURAL LANGUAGE PROCESSING.ipynb', 'sherlock_holmes.py']


In [37]:
# Run TFIDF and LDA for Modelling the Topics

import nltk, re
from sherlock_holmes import bohemia_ch1, bohemia_ch2, bohemia_ch3, boscombe_ch1, boscombe_ch2, boscombe_ch3 # Inputting the Data
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# preparing the text
corpus = [bohemia_ch1, bohemia_ch2, bohemia_ch3, boscombe_ch1, boscombe_ch2, boscombe_ch3]
preprocessed_corpus = [preprocess_text(chapter) for chapter in corpus]

# Update stop_list: - We Can Manually Update it Based on the Requirements/data we have
stop_list = ["say", "see", "holmes", "shall", "say", "man", "upon", "know", "quite", "one", 
"well", "could", "would", "take", "may", 
"think", "come", "go", "little", "must", 
"look"]
# filtering topics for stop words
def filter_out_stop_words(corpus):
    no_stops_corpus = []
    for chapter in corpus:
        no_stops_chapter = " ".join([word for word in chapter.split(" ") if word not in stop_list])
        no_stops_corpus.append(no_stops_chapter)
    return no_stops_corpus
filtered_for_stops = filter_out_stop_words(preprocessed_corpus)

# creating the bag of words model
bag_of_words_creator = CountVectorizer()
bag_of_words = bag_of_words_creator.fit_transform(filtered_for_stops)

# creating the tf-idf model
tfidf_creator = TfidfVectorizer(min_df = 0.2)
tfidf = tfidf_creator.fit_transform(preprocessed_corpus)

# LDA is a statistical model that takes your documents and determines which words keep popping up together in the same contexts (i.e., documents).

# creating the bag of words LDA model
lda_bag_of_words_creator = LatentDirichletAllocation(learning_method='online', n_components=10)
lda_bag_of_words = lda_bag_of_words_creator.fit_transform(bag_of_words)

# creating the tf-idf LDA model
lda_tfidf_creator = LatentDirichletAllocation(learning_method='online', n_components=10)
lda_tfidf = lda_tfidf_creator.fit_transform(tfidf)

print("~~~ Topics found by bag of words LDA ~~~")
for topic_id, topic in enumerate(lda_bag_of_words_creator.components_):
    message = "Topic #{}: ".format(topic_id + 1)
    message += " ".join([bag_of_words_creator.get_feature_names()[i] for i in topic.argsort()[:-5 :-1]])
    print(message)

print("\n\n~~~ Topics found by tf-idf LDA ~~~")
for topic_id, topic in enumerate(lda_tfidf_creator.components_):
    message = "Topic #{}: ".format(topic_id + 1)
    message += " ".join([tfidf_creator.get_feature_names()[i] for i in topic.argsort()[:-5 :-1]])
    print(message)

~~~ Topics found by bag of words LDA ~~~
Topic #1: hand cry room fire
Topic #2: mccarthy find mr father
Topic #3: father mccarthy young leave
Topic #4: find hear face mr
Topic #5: hand black leave room
Topic #6: throw mr sherlock majesty
Topic #7: majesty king photograph paper
Topic #8: nothing hand adler pull
Topic #9: son old cigar life
Topic #10: return mr find leave


~~~ Topics found by tf-idf LDA ~~~
Topic #1: convey stop life beside
Topic #2: adler work three interview
Topic #3: would arrest false wish
Topic #4: weapon precisely resolve kingdom
Topic #5: lawyer resolution within precisely
Topic #6: equally catch wonder peace
Topic #7: daughter child get find
Topic #8: holmes say man upon
Topic #9: hand power escape finally
Topic #10: five holmes kingdom repeat


### 6 - Text Similarity

Most of us have a good autocorrect story. Our phone’s messenger quietly swaps one letter for another as we type and suddenly the meaning of our message has changed (to our horror or pleasure). However, addressing text similarity — including spelling correction — is a major challenge within natural language processing.

Addressing word similarity and misspelling for spellcheck or autocorrect often involves considering the Levenshtein distance or minimal edit distance between two words. The distance is calculated through the minimum number of insertions, deletions, and substitutions that would need to occur for one word to become another. For example, turning “bees” into “beans” would require one substitution (“a” for “e”) and one insertion (“n”), so the Levenshtein distance would be two.

Phonetic similarity is also a major challenge within speech recognition. English-speaking humans can easily tell from context whether someone said “euthanasia” or “youth in Asia,” but it’s a far more challenging task for a machine! More advanced autocorrect and spelling correction technology additionally considers key distance on a keyboard and phonetic similarity (how much two words or phrases sound the same).

It’s also helpful to find out if texts are the same to guard against plagiarism, which we can identify through lexical similarity (the degree to which texts use the same vocabulary and phrases). Meanwhile, semantic similarity (the degree to which documents contain similar meaning or topics) is useful when you want to find (or recommend) an article or book similar to one you recently finished.

In [40]:
import nltk
# NLTK has a built-in function
# to check Levenshtein distance:
from nltk.metrics import edit_distance

def print_levenshtein(string1, string2):
    print("The Levenshtein distance from '{0}' to '{1}' is {2}!".format(string1, string2, edit_distance(string1, string2)))

# Check the distance between
# any two words here!
print_levenshtein("fart", "target")

# Assign passing strings here: - Get Levenshtein distance of 3
three_away_from_code = "decoder"

# Assign passing strings here: - Get Levenshtein distance of 3
two_away_from_chunk = "chunkey"

# Calculating Levenshtein Distance
print_levenshtein("code", three_away_from_code)
print_levenshtein("chunk", two_away_from_chunk)

The Levenshtein distance from 'fart' to 'target' is 3!
The Levenshtein distance from 'code' to 'decoder' is 3!
The Levenshtein distance from 'chunk' to 'chunkey' is 2!


### 7 - Language Prediction & Text Generation

How does your favorite search engine complete your search queries? How does your phone’s keyboard know what you want to type next? Language prediction is an application of NLP concerned with predicting text given preceding text. Autosuggest, autocomplete, and suggested replies are common forms of language prediction.

Your first step to language prediction is picking a language model. Bag of words alone is generally not a great model for language prediction; no matter what the preceding word was, you will just get one of the most commonly used words from your training corpus.

If you go the n-gram route, you will most likely rely on Markov chains to predict the statistical likelihood of each following word (or character) based on the training corpus. Markov chains are memory-less and make statistical predictions based entirely on the current n-gram on hand.

For example, let’s take a sentence beginning, “I ate so many grilled cheese”. Using a trigram model (where n is 3), a Markov chain would predict the following word as “sandwiches” based on the number of times the sequence “grilled cheese sandwiches” has appeared in the training data out of all the times “grilled cheese” has appeared in the training data.

A more advanced approach, using a neural language model, is the Long Short Term Memory (LSTM) model. LSTM uses deep learning with a network of artificial “cells” that manage memory, making them better suited for text prediction than traditional neural networks.

In [79]:
# 7 - Language Prediction / Text Generation Via Markov Chains

import nltk, re, random
from nltk.tokenize import word_tokenize
from collections import defaultdict, deque
from document1 import training_doc1
from document2 import training_doc2
from document3 import training_doc3

class MarkovChain:
    def __init__(self):
        self.lookup_dict = defaultdict(list)
        self._seeded = False
        self.__seed_me()

    def __seed_me(self, rand_seed=None):
        if self._seeded is not True:
            try:
                if rand_seed is not None:
                    random.seed(rand_seed)
                else:
                    random.seed()
                self._seeded = True
            except NotImplementedError:
                self._seeded = False
    
    def add_document(self, str):
        preprocessed_list = self._preprocess(str)
        pairs = self.__generate_tuple_keys(preprocessed_list)
        for pair in pairs:
            self.lookup_dict[pair[0]].append(pair[1])
  
    def _preprocess(self, str):
        cleaned = re.sub(r'\W+', ' ', str).lower()
        tokenized = word_tokenize(cleaned)
        return tokenized

    def __generate_tuple_keys(self, data):
        if len(data) < 1:
            return

        for i in range(len(data) - 1):
            yield [ data[i], data[i + 1] ]
      
    def generate_text(self, max_length=50):
        context = deque()
        output = []
        if len(self.lookup_dict) > 0:
            self.__seed_me(rand_seed=len(self.lookup_dict))
            chain_head = [list(self.lookup_dict)[0]]
            context.extend(chain_head)

            while len(output) < (max_length - 1):
                next_choices = self.lookup_dict[context[-1]]
                if len(next_choices) > 0:
                    next_word = random.choice(next_choices)
                    context.append(next_word)
                    output.append(context.popleft())
                else:
                    break
        output.extend(list(context))
        return " ".join(output)

my_markov = MarkovChain()
my_markov.add_document(training_doc1)
my_markov.add_document(training_doc2)
my_markov.add_document(training_doc3)
generated_text = my_markov.generate_text()
print(generated_text)
print(len(generated_text))

i m only a silver wing past the undertow just caught in the floods cross the truth that lies across this castle of being what i ve become so give me wrong to be more aware by becoming this castle of glass hardly anything there for you i want me
244


### 8 - Advanced NLP Topics

There are a slew of advanced topics and applications of NLP, many of which rely on deep learning and neural networks.

Naive Bayes classifiers are supervised machine learning algorithms that leverage a probabilistic theorem to make predictions and classifications. They are widely used for sentiment analysis (determining whether a given block of language expresses negative or positive feelings) and spam filtering.

We’ve made enormous gains in machine translation, but even the most advanced translation software using neural networks and LSTM still has far to go in accurately translating between languages.

Some of the most life-altering applications of NLP are focused on improving language accessibility for people with disabilities. Text-to-speech functionality and speech recognition have improved rapidly thanks to neural language models, making digital spaces far more accessible places.

NLP can also be used to detect bias in writing and speech. Feel like a political candidate, book, or news source is biased but can’t put your finger on exactly how? Natural language processing can help you identify the language at issue.

In [ ]:
# Here we Need Pickle files of Count_vectorizer and train, So Just View the Codes

import pickle
counter = pickle.load( open( "count_vect.p", "rb" ) )
training_counts =  pickle.load( open( "train.p", "rb" ) )

from reviews import counter, training_counts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Add your review:
review = "This Chapter is amazing, i think it is a Sentiment Classification Problem, but the course was marvelled and i was thrilled"
review_counts = counter.transform([review])

classifier = MultinomialNB()
training_labels = [0] * 1000 + [1] * 1000

classifier.fit(training_counts, training_labels)
  
neg = (classifier.predict_proba(review_counts)[0][0] * 100).round()
pos = (classifier.predict_proba(review_counts)[0][1] * 100).round()

if pos > 50:
    print("Thank you for your positive review!")
elif neg > 50:
    print("We're sorry this hasn't been the best possible lesson for you! We're always looking to improve.")
else:
    print("Naive Bayes cannot determine if this is negative or positive. Thank you or we're sorry?")

  
print("\nAccording to our trained Naive Bayes classifier, the probability that your review was negative was {0}% and the probability it was positive was {1}%.".format(neg, pos))

As you’ve seen, there are a vast array of applications for NLP. However, as they say, “with great language processing comes great responsibility” (or something along those lines). When working with NLP, we have several important considerations to take into account:

Different NLP tasks may be more or less difficult in different languages. Because so many NLP tools are built by and for English speakers, these tools may lag behind in processing other languages. The tools may also be programmed with cultural and linguistic biases specific to English speakers.

What if your Amazon Alexa could only understand wealthy men from coastal areas of the United States? English itself is not a homogeneous body. English varies by person, by dialect, and by many sociolinguistic factors. When we build and train NLP tools, are we only building them for one type of English speaker?

You can have the best intentions and still inadvertently program a bigoted tool. While NLP can limit bias, it can also propagate bias. As an NLP developer, it’s important to consider biases, both within your code and within the training corpus. A machine will learn the same biases you teach it, whether intentionally or unintentionally.

As you become someone who builds tools with natural language processing, it’s vital to take into account your users’ privacy. There are many powerful NLP tools that come head-to-head with privacy concerns. Who is collecting your data? How much data is being collected and what do those companies plan to do with your data?

Summary:

- Natural language processing combines computer science, linguistics, and artificial intelligence to enable computers to process human languages.
- NLTK is a Python library used for NLP.
- Text preprocessing is a stage of NLP focused on cleaning and preparing text for other NLP tasks.
- Parsing is a stage of NLP concerned with breaking up text based on syntax.
- Language models are probabilistic machine models of language use for NLP comprehension tasks. Common models include bag-of-words, n-gram models, and neural language modeling.
- Topic modeling is the NLP process by which hidden topics are identified given a body of text.
- Text similarity is a facet of NLP concerned with semblance between instances of language.
- Language prediction is an application of NLP concerned with predicting language given preceding language.
- There are many social and ethical considerations to take into account when designing NLP tools.

In [84]:
# Creating a Pilagarism Classifier

import nltk
# Levenshtein distance:
from nltk.metrics import edit_distance

# an arbitrary plagiarism classifier:
def is_plagiarized(text1, text2):
    n = 7
    if edit_distance(text1.lower(), text2.lower()) > ((len(text1) + len(text2)) / n):
        return False
    return True

doc1 = "is this plagiarized"
doc2 = "maybe it's plagiarized"
doc3 = "it's plagiarized"

print(is_plagiarized(doc1, doc2))
print(is_plagiarized(doc1, doc3))

False
True
